In [54]:
# Make imports
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
import numpy as np

In [2]:
# Read the data
# Read dataset file

DATA_DIRECTORY = "data"
INPUT_FILE = "OSA_DB_UPM.xlsx"

df_OSA = pd.read_excel(os.path.join(DATA_DIRECTORY, INPUT_FILE))

df_OSA.head()

,Patient,Gender,Smoker,AHI,BMI,Age,Cervical
0,P0002,hombre,si,29.6,39.305060,56,48.0
1,P0004,hombre,no,19.7,27.636054,39,42.0
2,P0005,hombre,no,9.0,26.729927,32,40.0
3,P0006,hombre,no,2.0,30.193906,32,42.0
4,P0007,hombre,no,34.0,30.110991,39,42.0


In [3]:
# Drop unused columns
df_OSA = df_OSA.drop(['Patient','Gender','Smoker'], axis=1)

df_OSA.head()

,AHI,BMI,Age,Cervical
0,29.6,39.305060,56,48.0
1,19.7,27.636054,39,42.0
2,9.0,26.729927,32,40.0
3,2.0,30.193906,32,42.0
4,34.0,30.110991,39,42.0


In [4]:
# Retrieve principal components

# Copy dataframe
df_OSA_PCA = df_OSA.copy()

# Drop response
df_OSA_PCA = df_OSA_PCA.drop(['AHI'], axis=1)

# Instantiate PCA with the 2 components
pca = PCA(n_components=2)

# Fit the PCA model to data
pca.fit(df_OSA_PCA)

# Transform the data to its principal components
components = pca.transform(df_OSA_PCA)

# Build dataframe of principal components
columns = [f'PC{i+1}' for i in range(2)]
df_PC = pd.DataFrame(data=components, columns=columns)

df_OSA = pd.concat([df_OSA, df_PC], axis=1)

df_OSA = df_OSA.drop(['BMI','Cervical','Age'], axis=1)

df_OSA.head()

,AHI,PC1,PC2
0,29.6,6.989714,11.588199
1,19.7,-10.464270,-0.910583
2,9.0,-17.542628,-2.406676
3,2.0,-17.397202,1.589114
4,34.0,-10.404591,1.264648


In [5]:
# Split dataframe into features and response
X = df_OSA[['PC1','PC2']]
y = df_OSA['AHI']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Function to performn GridSearch
def grid_searchCV(X_train, y_train, X_test, y_test, estimator, param_grid):
    # Create a GridSearchCV object
    grid_search = GridSearchCV(estimator=estimator,
                               param_grid=param_grid,
                               scoring='neg_mean_squared_error',
                               cv=5,
                               verbose=3)

    # Fit the model with grid search
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Evaluate on the test set
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error on Test Set:", mse)

    return best_model

In [7]:
# Polynomial regression

# Create a pipeline with PolynomialFeatures and LinearRegression
poly_reg = make_pipeline(PolynomialFeatures(), LinearRegression())

# Define the hyperparameter grid
param_grid_poly = {
    'polynomialfeatures__degree': [1, 2, 3, 4],
}

poly_reg_best = grid_searchCV(X_train,y_train,X_test,y_test,poly_reg,param_grid_poly)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ...polynomialfeatures__degree=1;, score=-217.540 total time=   0.0s
[CV 2/5] END ...polynomialfeatures__degree=1;, score=-367.265 total time=   0.0s
[CV 3/5] END ...polynomialfeatures__degree=1;, score=-280.299 total time=   0.0s
[CV 4/5] END ...polynomialfeatures__degree=1;, score=-324.892 total time=   0.0s
[CV 5/5] END ...polynomialfeatures__degree=1;, score=-191.883 total time=   0.0s
[CV 1/5] END ...polynomialfeatures__degree=2;, score=-228.274 total time=   0.0s
[CV 2/5] END ...polynomialfeatures__degree=2;, score=-367.007 total time=   0.0s
[CV 3/5] END ...polynomialfeatures__degree=2;, score=-284.700 total time=   0.0s
[CV 4/5] END ...polynomialfeatures__degree=2;, score=-313.904 total time=   0.0s
[CV 5/5] END ...polynomialfeatures__degree=2;, score=-196.567 total time=   0.0s
[CV 1/5] END ...polynomialfeatures__degree=3;, score=-239.925 total time=   0.0s
[CV 2/5] END ...polynomialfeatures__degree=3;, sc

In [45]:
# Polynomial regression - Ridge
poly_ridge = make_pipeline(PolynomialFeatures(), Ridge())

param_grid_ridge = {
    'polynomialfeatures__degree': [1, 2, 3, 4],
    'ridge__alpha': [0.1, 1, 10, 20]
}

poly_ridge_best = grid_searchCV(X_train,y_train,X_test,y_test,poly_ridge,param_grid_ridge)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END polynomialfeatures__degree=1, ridge__alpha=0.1;, score=-217.541 total time=   0.0s
[CV 2/5] END polynomialfeatures__degree=1, ridge__alpha=0.1;, score=-367.265 total time=   0.0s
[CV 3/5] END polynomialfeatures__degree=1, ridge__alpha=0.1;, score=-280.299 total time=   0.0s
[CV 4/5] END polynomialfeatures__degree=1, ridge__alpha=0.1;, score=-324.892 total time=   0.0s
[CV 5/5] END polynomialfeatures__degree=1, ridge__alpha=0.1;, score=-191.883 total time=   0.0s
[CV 1/5] END polynomialfeatures__degree=1, ridge__alpha=1;, score=-217.543 total time=   0.0s
[CV 2/5] END polynomialfeatures__degree=1, ridge__alpha=1;, score=-367.265 total time=   0.0s
[CV 3/5] END polynomialfeatures__degree=1, ridge__alpha=1;, score=-280.296 total time=   0.0s
[CV 4/5] END polynomialfeatures__degree=1, ridge__alpha=1;, score=-324.892 total time=   0.0s
[CV 5/5] END polynomialfeatures__degree=1, ridge__alpha=1;, score=-191.882 total ti

In [46]:
# Polynomial regression - Lasso
poly_lasso = make_pipeline(PolynomialFeatures(), Lasso())

param_grid_lasso = {
    'polynomialfeatures__degree': [1, 2, 3, 4], 
    'lasso__alpha': [0.1, 1, 10]
}

poly_lasso_best = grid_searchCV(X_train,y_train,X_test,y_test,poly_lasso,param_grid_lasso)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END lasso__alpha=0.1, polynomialfeatures__degree=1;, score=-217.609 total time=   0.0s
[CV 2/5] END lasso__alpha=0.1, polynomialfeatures__degree=1;, score=-367.296 total time=   0.0s
[CV 3/5] END lasso__alpha=0.1, polynomialfeatures__degree=1;, score=-280.194 total time=   0.0s
[CV 4/5] END lasso__alpha=0.1, polynomialfeatures__degree=1;, score=-324.907 total time=   0.0s
[CV 5/5] END lasso__alpha=0.1, polynomialfeatures__degree=1;, score=-191.855 total time=   0.0s
[CV 1/5] END lasso__alpha=0.1, polynomialfeatures__degree=2;, score=-228.379 total time=   0.0s
[CV 2/5] END lasso__alpha=0.1, polynomialfeatures__degree=2;, score=-367.071 total time=   0.0s
[CV 3/5] END lasso__alpha=0.1, polynomialfeatures__degree=2;, score=-284.602 total time=   0.0s
[CV 4/5] END lasso__alpha=0.1, polynomialfeatures__degree=2;, score=-313.902 total time=   0.0s
[CV 5/5] END lasso__alpha=0.1, polynomialfeatures__degree=2;, score=-196.47

In [66]:
# Create a Decision Tree Regressor
dt_regressor = DecisionTreeRegressor()

# Define the hyperparameter grid to search
param_grid_dt = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 80, 100, 120, 150]
}

dt_regressor_best = grid_searchCV(X_train,y_train,X_test,y_test,dt_regressor,param_grid_dt)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-431.382 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-525.218 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-599.482 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-523.069 total time=   0.0s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-435.645 total time=   0.0s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-382.944 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-492.031 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-569.807 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-531.587 total time= 

In [9]:
# Create an SVR (Support Vector Regression) model
svr_regressor = SVR()

# Define the hyperparameter grid to search
param_grid_svr = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [1, 2],
    'gamma': ['scale', 'auto']
}

svr_regressor_best = grid_searchCV(X_train,y_train,X_test,y_test,svr_regressor,param_grid_svr)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END C=0.1, degree=1, gamma=scale, kernel=linear;, score=-241.096 total time=   0.0s
[CV 2/5] END C=0.1, degree=1, gamma=scale, kernel=linear;, score=-386.782 total time=   0.0s
[CV 3/5] END C=0.1, degree=1, gamma=scale, kernel=linear;, score=-288.408 total time=   0.0s
[CV 4/5] END C=0.1, degree=1, gamma=scale, kernel=linear;, score=-348.354 total time=   0.0s
[CV 5/5] END C=0.1, degree=1, gamma=scale, kernel=linear;, score=-203.735 total time=   0.0s
[CV 1/5] END C=0.1, degree=1, gamma=scale, kernel=rbf;, score=-327.157 total time=   0.0s
[CV 2/5] END C=0.1, degree=1, gamma=scale, kernel=rbf;, score=-488.952 total time=   0.0s
[CV 3/5] END C=0.1, degree=1, gamma=scale, kernel=rbf;, score=-376.687 total time=   0.0s
[CV 4/5] END C=0.1, degree=1, gamma=scale, kernel=rbf;, score=-447.148 total time=   0.0s
[CV 5/5] END C=0.1, degree=1, gamma=scale, kernel=rbf;, score=-243.866 total time=   0.0s
[CV 1/5] END C=0.1, deg

In [10]:
# Nearest Neighbor
knn_regressor = KNeighborsRegressor()

param_grid_knn = {
    'n_neighbors': [3, 10, 20, 30, 50, 100],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

knn_regressor_best = grid_searchCV(X_train,y_train,X_test,y_test,knn_regressor,param_grid_knn)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END n_neighbors=3, p=1, weights=uniform;, score=-351.089 total time=   0.0s
[CV 2/5] END n_neighbors=3, p=1, weights=uniform;, score=-462.098 total time=   0.0s
[CV 3/5] END n_neighbors=3, p=1, weights=uniform;, score=-375.321 total time=   0.0s
[CV 4/5] END n_neighbors=3, p=1, weights=uniform;, score=-379.595 total time=   0.0s
[CV 5/5] END n_neighbors=3, p=1, weights=uniform;, score=-247.254 total time=   0.0s
[CV 1/5] END n_neighbors=3, p=1, weights=distance;, score=-356.711 total time=   0.0s
[CV 2/5] END n_neighbors=3, p=1, weights=distance;, score=-443.202 total time=   0.0s
[CV 3/5] END n_neighbors=3, p=1, weights=distance;, score=-350.590 total time=   0.0s
[CV 4/5] END n_neighbors=3, p=1, weights=distance;, score=-372.990 total time=   0.0s
[CV 5/5] END n_neighbors=3, p=1, weights=distance;, score=-275.937 total time=   0.0s
[CV 1/5] END n_neighbors=3, p=2, weights=uniform;, score=-367.827 total time=   0.0

In [12]:
# Create a Random Forest regressor
rf_regressor = RandomForestRegressor()

# Define the parameter grid for grid search
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_regressor_best = grid_searchCV(X_train,y_train,X_test,y_test,rf_regressor,param_grid_rf)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV 1/5] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-284.396 total time=   0.1s
[CV 2/5] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-359.192 total time=   0.1s
[CV 3/5] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-322.496 total time=   0.1s
[CV 4/5] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-343.530 total time=   0.1s
[CV 5/5] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-249.230 total time=   0.1s
[CV 1/5] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-280.802 total time=   0.2s
[CV 2/5] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, sco

In [63]:
# Train with whole dataset
tuned_poly_reg = make_pipeline(PolynomialFeatures(degree=1), LinearRegression())

final_poly_reg = tuned_poly_reg.fit(X_train,y_train)

In [64]:
tuned_ridge_reg = make_pipeline(PolynomialFeatures(degree=1), Ridge(alpha=20))

final_ridge_reg = tuned_ridge_reg.fit(X_train,y_train)

In [65]:
tuned_poly_lasso = make_pipeline(PolynomialFeatures(degree=1), Lasso(alpha=1))

final_lasso_reg = tuned_poly_lasso.fit(X_train,y_train)

In [67]:
tuned_dt = DecisionTreeRegressor(max_depth=15,min_samples_leaf=80,min_samples_split=10)

final_dt = tuned_dt.fit(X_train,y_train)

In [68]:
tuned_svr = SVR(C=1, degree=1, gamma='scale', kernel='linear')

final_svr = tuned_svr.fit(X_train,y_train)

In [69]:
tuned_knn = KNeighborsRegressor(n_neighbors=30, p=1, weights='uniform')

final_knn = tuned_knn.fit(X_train, y_train)

In [70]:
tuned_rf = RandomForestRegressor(bootstrap=True,
                                 max_depth=None,
                                 min_samples_leaf=4,
                                 min_samples_split=10,
                                 n_estimators=100)

final_rf = tuned_rf.fit(X_train, y_train)

In [42]:
def print_errors(mse, mae):
    print("\tMSE: " + str(mse))
    print("\tRMSE: " + str(math.sqrt(mse)))
    print("\tMAE: " + str(mae))
    print("")

def measure_error(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)

    y_test_pred = model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)

    print("--- Training Set ---")
    print_errors(mse_train, mae_train)

    print("--- Test Set ---")
    print_errors(mse_test, mae_test)


In [71]:
print("Polynomial regression")
measure_error(final_poly_reg,X_train,y_train,X_test,y_test)
print("-----------------------------------")
print("Ridge")
measure_error(final_ridge_reg,X_train,y_train,X_test,y_test)
print("-----------------------------------")
print("Lasso")
measure_error(final_lasso_reg,X_train,y_train,X_test,y_test)
print("-----------------------------------")
print("Decision Tree")
measure_error(final_dt,X_train,y_train,X_test,y_test)
print("-----------------------------------")
print("SVR")
measure_error(final_svr,X_train,y_train,X_test,y_test)
print("-----------------------------------")
print("KNN")
measure_error(final_knn,X_train,y_train,X_test,y_test)
print("-----------------------------------")
print("Random Forest")
measure_error(final_rf,X_train,y_train,X_test,y_test)

Polynomial regression
--- Training Set ---
	MSE: 274.7357616136879
	RMSE: 16.575154949914886
	MAE: 12.500893545286466

--- Test Set ---
	MSE: 289.1184789905149
	RMSE: 17.003484319118684
	MAE: 13.05766897469384

-----------------------------------
Ridge
--- Training Set ---
	MSE: 274.73583712199337
	RMSE: 16.57515722767037
	MAE: 12.500881523385088

--- Test Set ---
	MSE: 289.0817489635896
	RMSE: 17.002404211275227
	MAE: 13.056724244971313

-----------------------------------
Lasso
--- Training Set ---
	MSE: 274.768351526101
	RMSE: 16.57613801601872
	MAE: 12.50613246773119

--- Test Set ---
	MSE: 288.2878152492997
	RMSE: 16.979040469040047
	MAE: 13.037216974137337

-----------------------------------
Decision Tree
--- Training Set ---
	MSE: 281.8125056589633
	RMSE: 16.787272132748765
	MAE: 12.4634541092375

--- Test Set ---
	MSE: 306.72946054941957
	RMSE: 17.51369351534449
	MAE: 13.666705435810355

-----------------------------------
SVR
--- Training Set ---
	MSE: 291.6025455552248
	RMSE

In [61]:
y_test.shape

(125,)